<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/Auswertung_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Verbinden mit der Google-Drive
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
import pandas as pd

In [8]:
def parse_model_log(txt_path):
    """
    Liest die Logdatei ein und erstellt eine Tabelle (pd.DataFrame),
    bei der jeder Block einen Eintrag repräsentiert.
    """
    data = []           # Hier sammeln wir alle Einträge (als dict)
    current_entry = {}  # Temporäres Dictionary für den aktuellen Block
    in_entry = False    # Flag, ob wir uns aktuell in einem "=== Model Results ==="-Block befinden

    if not os.path.exists(txt_path):
        raise FileNotFoundError(f"Die angegebene Datei {txt_path} existiert nicht.")

    with open(txt_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # Blockanfang
            if line.startswith("=== Model Results ==="):
                current_entry = {}
                in_entry = True
                continue

            # Blockende
            if line.startswith("======================"):
                if in_entry and current_entry:
                    data.append(current_entry)
                in_entry = False
                current_entry = {}
                continue

            # Innerhalb des Blocks: Zeilen "key: value" verarbeiten
            if in_entry and line:
                # Erwartete Struktur: "key: value"
                if ": " in line:
                    key, value = line.split(": ", 1)
                    current_entry[key] = value
                # Falls Zeilen ohne ": " vorkommen, kannst du sie ggf. überspringen
                # else:
                #    pass

    # Jetzt haben wir eine Liste aus Dictionaries, jeder entspricht einem Log-Block
    df = pd.DataFrame(data)
    return df

In [11]:
# Beispiel: Lies aus dem Drive
txt_file_path = "/content/drive/MyDrive/Colab_Notebooks/LSTM_Model/model_results.txt"

df_results = parse_model_log(txt_file_path)

=== Ergebnisse aus der Log-Datei ===
      model_name model_config.units_1 model_config.units_2  \
0  Baseline LSTM                   64                   32   

  model_config.dropout_rate model_config.final_activation  \
0                       0.1                        linear   

  training_config.loss training_config.optimizer  \
0   mean_squared_error                      adam   

                             training_config.metrics fit_config.epochs  \
0  ['mse', 'mae', 'mape', <RootMeanSquaredError n...                 5   

  fit_config.batch_size  ...              val_loss               val_mse  \
0                    32  ...  0.007743313908576965  0.007743313908576965   

               val_mae           val_rmse            test_loss  \
0  0.05897708237171173  44.57095718383789  0.00892695877701044   

              test_mse             test_mae           test_rmse  \
0  0.00892695877701044  0.06294387578964233  60.592220306396484   

               val_r2             test_r

In [12]:
df_results

,model_name,model_config.units_1,model_config.units_2,model_config.dropout_rate,model_config.final_activation,training_config.loss,training_config.optimizer,training_config.metrics,fit_config.epochs,fit_config.batch_size,...,val_loss,val_mse,val_mae,val_rmse,test_loss,test_mse,test_mae,test_rmse,val_r2,test_r2
0,Baseline LSTM,64,32,0.1,linear,mean_squared_error,adam,"['mse', 'mae', 'mape', <RootMeanSquaredError n...",5,32,...,0.007743313908576965,0.007743313908576965,0.05897708237171173,44.57095718383789,0.00892695877701044,0.00892695877701044,0.06294387578964233,60.592220306396484,0.7900590413070319,0.7836071803082321
